# Location

The `location` directive, used within the `server` context, specifies how NGINX should process requests based on the URI.

More in [correspoding section](http://nginx.org/en/docs/http/ngx_http_core_module.html#location) of the official documentation.

## Setup

It turns out that it's quite a complex task to build examples that show how everything works, so this section describes what we need to show everything. In summary, we need

- Proxied server - server where we'll redirect requests to nginx.
- And nginx, which can be configured differently for different examples.

### Proxied server

Here is the simplest possible http server in Python. It's purpose is just to return it's input as raw text - so we can check what was sent to the server.

In [7]:
%%writefile location_files/proxy_set_header.py
import http.server
import socketserver

class RequestHandler(http.server.BaseHTTPRequestHandler):
    def do_GET(self):
        self.send_response(200)
        self.send_header("Content-type", "text/plain")
        self.end_headers()
        self.wfile.write(b"Server received your GET request\n")
        self.wfile.write(b"Raw request data:\n\n")
        self.wfile.write(self.raw_requestline)
        self.wfile.write(b"\nHeaders:\n")
        self.wfile.write(bytes(str(self.headers), "utf-8"))

PORT = 7890
with socketserver.TCPServer(("", PORT), RequestHandler) as httpd:
    print("Serving at port", PORT)
    httpd.serve_forever()

Overwriting location_files/proxy_set_header.py


For standardisation purposes, we'll run this server in the docker container.

In [16]:
!docker run --rm -itd -p 7890:7890 \
    --name destination_server \
    -v $(pwd)/location_files/proxy_set_header.py:/proxy_set_header.py \
    python:3.10-alpine python proxy_set_header.py

0b8ae6b03af7745cd6c48ad5840f1927137cb23a0ca65f0e34825118504ed8ff


The next cell shows how it works - it sends a request to the server we created. And for example, we'll add some headers to the request.

In [25]:
import requests
print(
    requests.get(
        "http://localhost:7890/my_data", 
        headers={"my_field": "my_value"}
    ).content.decode("utf-8")
)

Server received your GET request
Raw request data:

GET /my_data HTTP/1.1

Headers:
Host: localhost:7890
User-Agent: python-requests/2.31.0
Accept-Encoding: gzip, deflate
Accept: */*
Connection: keep-alive
my_field: my_value




As a result we got raw GET HTTP request and there is defined by us header field in the output.

### Nginx

In [4]:
%%writefile location_files/nginx.conf
events {}

http {
    server {
        listen 80;
        location / {proxy_pass "https://google.com/";}
    }
}

Overwriting location_files/nginx.conf


In [6]:
!docker run -itd --rm -p 80:80 --name nginx_location \
    -v $(pwd)/location_files/nginx.conf:/etc/nginx/nginx.conf nginx

b45a4acb471835e9df1afb43ba96cccfc7b5f8b241db4f8505c6734e40caf06a


In [ ]:
!docker stop nginx_location

## Headers to proxied server (`proxy_set_header`)

Allows redefining or appending fields to the request header passed to the proxied server. You can add aditional headers to the http request that will be sent to the destination server.